In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import pyproj
import warnings
import os

In [2]:
os.environ['HDF5_USE_FILE_LOCKING']='FALSE'

In [3]:
def uv_to_earth_kyle(lon, lat, proj, u, v):
    m = proj.get_factors(lon, lat)

    dy_dlam = m.dy_dlam
    dx_dlam = m.dx_dlam

    theta = -np.arctan2(dy_dlam, dx_dlam)

    return (
        u * np.cos(theta) - v * np.sin(theta),
        u * np.sin(theta) + v * np.cos(theta),
    )


def uv_to_grid_kyle(lon, lat, proj, u, v):
    m = proj.get_factors(lon, lat)

    dy_dlam = m.dy_dlam
    dx_dlam = m.dx_dlam

    theta = np.arctan2(dy_dlam, dx_dlam)

    return (
        u * np.cos(theta) - v * np.sin(theta),
        u * np.sin(theta) + v * np.cos(theta),
    )


def get_nearest_point(projection, longitude, latitude):
    # Bill's WRF domain
    lon_c, lat_c = -129., 37.
    nx, ny = 699, 699
    half_n = nx//2
    dx = 3000. # m
    xa = np.arange(-half_n*dx, half_n*dx + dx, dx)
    ya = np.arange(-half_n*dx, half_n*dx + dx, dx)

    pc = ccrs.PlateCarree()
    # x, y = projection.transform_point(longitude, latitude, ccrs.PlateCarree())

    proj_2 = pyproj.Transformer.from_crs(pc, projection)

    x, y = proj_2.transform(longitude, latitude)
    x_c, y_c = proj_2.transform(lon_c, lat_c)

    xi = np.argmin(np.abs(x - x_c - xa))
    yi = np.argmin(np.abs(y - y_c - ya))

    return xi, yi

In [4]:
globe = ccrs.Globe(ellipse="sphere", semimajor_axis=6370000, semiminor_axis=6370000)

# map projections for WRF
''' Copied from WRF output file
                :CEN_LAT = 37.00002f ;
                :CEN_LON = -129.f ;
                :TRUELAT1 = 30.f ;
                :TRUELAT2 = 42.f ;
                :MOAD_CEN_LAT = 37.00002f ;
                :STAND_LON = -100.f ;
'''
crsproj_wrf = ccrs.LambertConformal(
    central_longitude=-100.,
    central_latitude=37.,
    standard_parallels=[30., 42.],
    globe=globe,
)
pyproj_wrf = pyproj.Proj(crsproj_wrf)

# map projections for PINACLES
lat_center = 35.00
lon_center = -123.71
proj_lon_center = -68.0
crsproj_pinacles = ccrs.LambertConformal(
    central_longitude=proj_lon_center,
    central_latitude=lat_center,
    standard_parallels=(lat_center, lat_center),
    globe=globe,
)
pyproj_pinacles = pyproj.Proj(crsproj_pinacles)


In [5]:
# DATES = pd.date_range("2021-03-09 12:00", "2021-03-11 00:00", freq="30T")
DATES = pd.date_range("2021-03-09 12:00", "2021-03-09 13:00", freq="30T")
start_time = DATES[0]
start_tstring = start_time.strftime("%Y-%m-%d_%H:%M:%S")

In [9]:
for DATE in DATES:
    datestring = DATE.strftime("%Y-%m-%d_%H_%M_%S")
    in_file = f"/lustre/eaglefs/projects/lidarbuoy/d3m088/wrf/era5/run_wrf_20210309/output_morr/wrf2pin_d01_{datestring}"
    out_file = (
        f"/lustre/eaglefs/scratch/xiao169/pinacles_in.wrf/pinacles_in_{datestring}.nc"
    )
    print(f"Input: {in_file}")
    print(f"Output: {out_file}")
    ds = xr.open_dataset(in_file, decode_times=False)
    new_unit = f"minutes since {start_tstring}"
    ds.XTIME.attrs["description"] = new_unit
    ds.XTIME.attrs["units"] = new_unit
    ds = xr.decode_cf(ds)
    print(f"Time: {ds.XTIME.values[0]}")
    if DATE == start_time:
        width = 100
        xi, yi = get_nearest_point(crsproj_wrf, lon_center, lat_center)
        xslice = slice(xi - width, xi + width)
        yslice = slice(yi - width, yi + width)
        print(
            f"PINACLES domain center on WRF grid, lon = {ds.XLONG.values[0, yi, xi]}, lat = {ds.XLAT.values[0, yi, xi]}"
        )
        print("Slices of HRRR domain to extract:")
        print(xslice, yslice)
    ds = ds.rename_dims(
        {"west_east": "x", "south_north": "y", "Time": "t", "bottom_top": "z"}
    )
    ds = ds.rename_vars(
        {
            "XTIME": "time",
            "XLAT": "latitude",
            "XLONG": "longitude",
            "Q2": "QV2m",
            "T2": "T2m",
            "QVAPOR": "QV",
            "QCLOUD": "QC",
            "QICE": "QI",
            "U10": "U10m",
            "V10": "V10m",
        }
    )
    # ds = ds.reset_coords(["XLAT", "XLONG"])
    ds['latitude'] = xr.DataArray(data=ds.latitude[0,:,:], dims=['y', 'x'], attrs={"units":"degree_north", "description":"LATITUDE, SOUTH IS NEGATIVE" })
    ds['longitude'] = xr.DataArray(data=ds.longitude[0,:,:], dims=['y', 'x'], attrs={"units":"degree_east", "description":"LONGITUDE, WEST IS NEGATIVE"})
    # ds.set_coords(['latitude', 'longitude'])
    print(ds)
    dout = xr.Dataset()
    # 2D: SST, PSFC, QV2m, T2m, U10m, V10m
    dout["SST"] = ds.SST.isel(x=xslice, y=yslice)
    dout["PSFC"] = ds.PSFC.isel(x=xslice, y=yslice)
    dout["QV2m"] = ds.QV2m.isel(x=xslice, y=yslice)
    dout["T2m"] = ds.T2m.isel(x=xslice, y=yslice)
    u10m, v10m = ds.U10m.isel(x=xslice, y=yslice), ds.V10m.isel(x=xslice, y=yslice)
    print(u10m)
    lon = u10m.longitude.values
    lat = u10m.latitude.values
    # Rotate wind into PINACLES Projection
    u_earth, v_earth = uv_to_earth_kyle(lon, lat, pyproj_wrf, u10m.values, v10m.values)
    u10m.values, v10m.values = uv_to_grid_kyle(
        lon, lat, pyproj_pinacles, u_earth, v_earth
    )
    dout["U10m"] = u10m
    dout["V10m"] = v10m
    # 3D: QV, QC, QI, U, V, P, T, Z
    dout["QV"] = ds.QV.isel(x=xslice, y=yslice)
    dout["QC"] = ds.QC.isel(x=xslice, y=yslice)
    dout["QI"] = ds.QI.isel(x=xslice, y=yslice)
    ui, vi = ds.U.values, ds.V.values
    u = (ui[:, :, :, 1:] + ui[:, :, :, :-1]) * 0.5
    v = (vi[:, :, 1:, :] + vi[:, :, :-1, :]) * 0.5
    u_earth, v_earth = uv_to_earth_kyle(
        lon, lat, pyproj_wrf, u[:, :, yslice, xslice], v[:, :, yslice, xslice]
    )
    u[:, :, yslice, xslice], v[:, :, yslice, xslice] = uv_to_grid_kyle(
        lon, lat, pyproj_pinacles, u_earth, v_earth
    )
    u_dressed = xr.DataArray(
        data=u, dims=["time", "z", "y", "x"], attrs={"units": "m/s"}
    )
    v_dressed = xr.DataArray(
        data=v, dims=["time", "z", "y", "x"], attrs={"units": "m/s"}
    )
    dout["U"] = u_dressed.isel(x=xslice, y=yslice)
    dout["V"] = v_dressed.isel(x=xslice, y=yslice)
    p_dressed = xr.DataArray(data=ds.P.values+ds.PB.values, dims=["time", "z", "y", "x"], attrs={"units": "Pa"})
    dout["P"] = p_dressed.isel(x=xslice, y=yslice)
    g = 9.81
    z = (ds.PH.values[:,:-1,:,:]+ds.PHB.values[:,1:,:,:])*0.5/g
    z_dressed = xr.DataArray(data=z, dims=["time", "z", "y", "x"], attrs={"units": "m"}) 
    dout["Z"] = z_dressed.isel(x=xslice, y=yslice)
    rd = 287.
    rv = 461.6 
    t0 = 300. # K
    t = (ds.THM.values + t0)/(1.+rv*ds.QV.values/rd)
    t_dressed = xr.DataArray(data=t, dims=["time", "z", "y", "x"], attrs={"units": "K"}) 
    dout["T"] = t_dressed.isel(x=xslice, y=yslice)
    try:
        dout.to_netcdf(out_file)
    except PermissionError:
        print("AGAIN?")
    except:
        print("What Happened?")
    else:
        print("Good!")
    finally:
        dout.close()
        ds.close()
        del ds, dout
        xr.backends.file_manager.FILE_CACHE.clear()

Input: /lustre/eaglefs/projects/lidarbuoy/d3m088/wrf/era5/run_wrf_20210309/output_morr/wrf2pin_d01_2021-03-09_12_00_00
Output: /lustre/eaglefs/scratch/xiao169/pinacles_in.wrf/pinacles_in_2021-03-09_12_00_00.nc
Time: 2021-03-09T12:00:00.000000000
PINACLES domain center on WRF grid, lon = -123.71951293945312, lat = 35.00368118286133
Slices of HRRR domain to extract:
slice(381, 581, None) slice(136, 336, None)
<xarray.Dataset>
Dimensions:    (t: 15, y: 699, x: 699, z: 129, west_east_stag: 700,
                south_north_stag: 700, bottom_top_stag: 130)
Coordinates:
    latitude   (y, x) float32 25.02 25.02 25.03 25.04 ... 48.42 48.43 48.43
    longitude  (y, x) float32 -135.8 -135.8 -135.7 ... -119.8 -119.8 -119.8
    time       (t) datetime64[ns] ...
Dimensions without coordinates: t, y, x, z, west_east_stag, south_north_stag,
                                bottom_top_stag
Data variables: (12/21)
    Times      (t) |S19 ...
    U          (t, z, y, west_east_stag) float32 ...
    V    